In [7]:
import pandas as pd
import plotly.express as px

In [18]:
df_principal = pd.read_excel('/content/Imersão Python - Tabela de ações.xlsx', sheet_name='Principal')
df_principal.head(10)

FileNotFoundError: [Errno 2] No such file or directory: 'Imersão Python - Tabela de ações.xlsx'

In [ ]:
df_total_de_acoes = pd.read_excel('/content/Imersão Python - Tabela de ações.xlsx', sheet_name='Total_de_acoes')
df_total_de_acoes.head(10)

In [ ]:
df_ticker = pd.read_excel('/content/Imersão Python - Tabela de ações.xlsx', sheet_name='Ticker')
df_ticker.head(10)

In [ ]:
df_chatgpt = pd.read_excel('/content/Imersão Python - Tabela de ações.xlsx', sheet_name='ChatGPT')
df_chatgpt.head(10)

In [ ]:
df_principal = df_principal[['Ativo','Data','Último (R$)','Var. Dia (%)']].copy()
df_principal

In [ ]:
df_principal = df_principal.rename(columns = {'Último (R$)':'Valor_final', 'Var. Dia (%)':'Var_dia_pct'}).copy()
df_principal

In [ ]:
df_principal['Var_pct'] = df_principal['Var_dia_pct'] / 100
df_principal

In [ ]:
df_principal['Var_inicial'] = df_principal['Valor_final'] / (df_principal['Var_pct'] + 1)
df_principal

In [ ]:
#Mesma função que o VLOOKUP do excel
df_principal = df_principal.merge(df_total_de_acoes, left_on='Ativo', right_on='Código', how='left')
df_principal

In [ ]:
df_principal = df_principal.drop(columns=['Código'])
df_principal

In [ ]:
df_principal['Variacao_rs'] = (df_principal['Valor_final'] - df_principal['Var_inicial']) * df_principal['Qtde. Teórica']
df_principal

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
df_principal

In [ ]:
df_principal['Qtde. Teórica'] = df_principal['Qtde. Teórica'].astype(int)
df_principal

In [ ]:
df_principal = df_principal.rename(columns = {'Qtde. Teórica':'Qtd_teorica'}).copy()
df_principal

In [ ]:
df_principal['Resultado'] = df_principal['Variacao_rs'].apply(lambda x: 'Subiu' if x > 0 else ('Desceu' if x < 0 else 'Estável'))
df_principal

In [ ]:
#Mesma função que o VLOOKUP do excel
df_principal = df_principal.merge(df_ticker, left_on='Ativo', right_on='Ticker', how='left')
df_principal = df_principal.drop(columns=['Ticker'])
df_principal

In [ ]:
#Mesma função que o VLOOKUP do excel
df_principal = df_principal.merge(df_chatgpt, left_on='Nome', right_on='Nome da Empresa', how='left')
df_principal = df_principal.drop(columns=['Nome da Empresa'])
df_principal

In [ ]:
df_principal['Cat_idade'] = df_principal['Idade em Anos'].apply(lambda x: 'Mais de 100' if x > 100 else ('Menos de 50' if x < 50 else 'Entre 50 e 100'))
df_principal

In [ ]:
#Calculando o maior valor
maior = df_principal['Variacao_rs'].max()

#Calculando o menor valor
menor = df_principal['Variacao_rs'].min()

#Calculando a média
media = df_principal['Variacao_rs'].mean()

#Calculando a média de quem subiu
media_subiu = df_principal[df_principal['Resultado'] == 'Subiu']['Variacao_rs'].mean()

#Calculando a média de quem desceu
media_desceu = df_principal[df_principal['Resultado'] == 'Desceu']['Variacao_rs'].mean()

#Imprimindo os resultados
print(f'Maior\tR$ {maior:,.2f}')
print(f'Menor\tR$ {menor:,.2f}')
print(f'Média\tR$ {media:,.2f}')
print(f'Média de quem subiu\tR$ {media_subiu:,.2f}')
print(f'Média de quem desceu\tR$ {media_desceu:,.2f}')

In [ ]:
df_principal_subiu = df_principal[df_principal['Resultado'] == 'Subiu']
df_principal_subiu

In [ ]:
#colocando o reset_index, ele vai continuar como dataframe, se não colocar, ele cria o objeto como vetor
df_analise_segmento = df_principal_subiu.groupby('Segmento')['Variacao_rs'].sum().reset_index()
df_analise_segmento

In [ ]:
#colocando o reset_index, ele vai continuar como dataframe, se não colocar, ele cria o objeto como vetor
df_analise_saldo = df_principal.groupby('Resultado')['Variacao_rs'].sum().reset_index()
df_analise_saldo

# Gráfico

In [ ]:
fig_analise_saldo = px.bar(df_analise_saldo, x='Resultado', y='Variacao_rs', text='Variacao_rs', title='Variação Reais por Resultado')
fig_analise_saldo.show()

In [ ]:
fig_analise_segmento = px.pie(df_analise_segmento, names='Segmento', values='Variacao_rs', title='Variação Reais por Segmento')
fig_analise_segmento.show()

In [ ]:
df_analise_cat_idade = df_principal.groupby('Cat_idade')['Variacao_rs'].sum().reset_index()
df_analise_cat_idade

fig_analise_cat_idade = px.bar(df_analise_cat_idade, x='Cat_idade', y='Variacao_rs', text='Variacao_rs', title='Variação Reais por Categoria de Idade')
fig_analise_cat_idade.show()